In [24]:
import numpy as np
import pandas as pd
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList, cuda
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
from chaineripy.extensions import PrintReport, ProgressBar
import sqlite3

db_name = 'db.sqlite3'
con = sqlite3.connect(db_name)

In [2]:
def zs(p, n):
    return (p - p.rolling(n).mean()) / p.rolling(n).std()


def avg(p, n):
    return p.rolling(n).mean()


def std(p, n):
    return p.rolling(n).std()

In [4]:
db_name = 'db.sqlite3'
con = sqlite3.connect(db_name)
print('loading from db')
df = pd.read_sql_query('select * from history', con)
print('loaded from db')
df.exec_date = pd.to_datetime(df.exec_date)
df = df.set_index('exec_date')
df = df.loc['2016-03':]
df = df[['id', 'side', 'price', 'size']]
bench_price = df.price.resample('15Min').mean().fillna(method='ffill')
bench_size = df['size'].resample('15Min').sum().fillna(0)
dfb = pd.DataFrame([bench_price, bench_size]).T

p = dfb.price
v = dfb['size']

dfb['pma12'] = zs(p / avg(p, 12) - 1, 96)
dfb['pma96'] = zs(p / avg(p, 96) - 1, 96)
dfb['pma672'] = zs(p / avg(p, 672) - 1, 96)

dfb['ma4_36'] = zs(avg(p, 4) / avg(p, 36) - 1, 96)
dfb['ma12_96'] = zs(avg(p, 12) / avg(p, 96) - 1, 96)
dfb['ac12_12'] = zs((p / avg(p, 12)) / avg(p / avg(p, 12), 12), 96)
dfb['ac96_96'] = zs((p / avg(p, 96)) / avg(p / avg(p, 96), 12), 96)

dfb['vma12'] = zs(v / avg(v, 12) - 1, 96)
dfb['vma96'] = zs(v / avg(v, 96) - 1, 96)
dfb['vma672'] = zs(v / avg(v, 672) - 1, 96)

dfb['vZ12'] = zs(v, 12)
dfb['vZ96'] = zs(v, 96)
dfb['vZ672'] = zs(v, 672)

r_label_list = [
    'r', 'r_1', 'r_2',
]
for r_label in r_label_list:
    dfb[r_label] = 0
dfb['state'] = 0

dfb['pZ12'] = zs(p, 12)
dfb['pZ96'] = zs(p, 96)
dfb['vol12'] = zs(std(p, 12), 96)
dfb['vol96'] = zs(std(p, 96), 96)
dfb['vol672'] = zs(std(p, 672), 96)
dfb['dv12_96'] = zs(std(p, 12) / avg(std(p, 12), 96), 96)
dfb['dv96_672'] = zs(std(p, 96) / avg(std(p, 96), 672), 96)
dfb['utctime'] = (dfb.index.hour * 4 + dfb.index.minute / 15) / 96

dfz = dfb

indicator = dfz.reset_index().loc[:, 'pma12':'utctime']
price_history = dfz.reset_index().price

indicator['vma12'] = indicator.vma12.fillna(0)
indicator['vZ12'] = indicator.vZ12.fillna(0)

loading from db
loaded from db


In [5]:
from matplotlib import pyplot as plt
%matplotlib inline

In [7]:
!pip install chaineripy

In [8]:
indicator

,pma12,pma96,pma672,ma4_36,ma12_96,ac12_12,ac96_96,vma12,vma96,vma672,...,r_2,state,pZ12,pZ96,vol12,vol96,vol672,dv12_96,dv96_672,utctime
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.010417
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.020833
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.031250
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.041667
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.052083
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.062500
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.072917
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.083333
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.093750


In [14]:
answer = price_history / price_history.shift() - 1
answer = (answer > 0.0005) * 1 - (answer < -0.0005) * 1 + 1
answer

0        1
1        2
2        2
3        2
4        0
5        2
6        1
7        1
8        1
9        0
10       2
11       1
12       2
13       0
14       1
15       1
16       0
17       1
18       1
19       1
20       0
21       2
22       1
23       0
24       2
25       0
26       2
27       1
28       2
29       1
        ..
65182    2
65183    2
65184    0
65185    1
65186    0
65187    2
65188    2
65189    1
65190    0
65191    2
65192    1
65193    1
65194    1
65195    2
65196    2
65197    2
65198    1
65199    0
65200    0
65201    2
65202    2
65203    2
65204    1
65205    0
65206    0
65207    0
65208    2
65209    2
65210    0
65211    0
Name: price, Length: 65212, dtype: int32

In [15]:
price_history / price_history.shift() - 1

0             NaN
1        0.000742
2        0.000571
3        0.002030
4       -0.004401
5        0.000611
6       -0.000234
7       -0.000330
8       -0.000062
9       -0.003218
10       0.000804
11       0.000115
12       0.001355
13      -0.001247
14       0.000018
15      -0.000060
16      -0.000543
17       0.000206
18      -0.000367
19      -0.000339
20      -0.001607
21       0.001986
22      -0.000110
23      -0.000602
24       0.000579
25      -0.000843
26       0.000954
27      -0.000149
28       0.001603
29      -0.000484
           ...   
65182    0.006057
65183    0.005119
65184   -0.001073
65185    0.000358
65186   -0.001072
65187    0.004256
65188    0.008042
65189   -0.000361
65190   -0.001405
65191    0.004955
65192   -0.000015
65193   -0.000344
65194   -0.000015
65195    0.004789
65196    0.003887
65197    0.003343
65198    0.000101
65199   -0.004524
65200   -0.005368
65201    0.006042
65202    0.002052
65203    0.005172
65204   -0.000067
65205   -0.001911
65206   -0

In [16]:
answer[answer == 0].size, answer[answer == 1].size, answer[answer == 2].size

(20149, 22703, 22360)

In [17]:
indexer = indicator.dv96_672.notnull()

In [30]:
xp = cuda.cupy

d_exp = indicator.loc[indexer]
d_obj = answer[indexer]
data = xp.array(d_exp, dtype=xp.float32)
t_data = xp.array(d_obj, dtype=xp.int32)

In [31]:
row, col = d_exp.shape

ls_1 = 500
ls_2 = 1000
ls_3 = 200
out_size = 3
class MyChain(Chain):
    def __init__(self):
        super().__init__(
            l1=L.Linear(col, ls_1),
            l2=L.Linear(ls_1, ls_2),
            l3=L.Linear(ls_2, ls_3),
            l4=L.Linear(ls_3, out_size)
        )
    
    def __call__(self, x):
        h1 = F.sigmoid(self.l1(x))
        h2 = F.sigmoid(self.l2(h1))
        h3 = F.sigmoid(self.l3(h2))
        o = self.l4(h3)
        return o

In [32]:
train = datasets.tuple_dataset.TupleDataset(data, t_data)
train = iterators.SerialIterator(train, batch_size=100, shuffle=True, repeat=True)

In [33]:
model = L.Classifier(MyChain())

gpu_device = 0
cuda.get_device_from_id(gpu_device).use()
model.to_gpu(gpu_device)

optimizer = optimizers.SGD()
optimizer.setup(model)
updater = training.StandardUpdater(train, optimizer)

In [34]:
trainer = training.Trainer(updater, (30, 'epoch'), out='result')
#精度確認
# trainer.extend(extensions.Evaluator(test_iter, model))
#レポート出力
trainer.extend(extensions.LogReport())
# print the statistics to a IPython widget
trainer.extend(PrintReport(
    ['epoch', 'main/loss', 'validation/main/loss',
     'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))
# show the progress bar on a IPython widget
trainer.extend(ProgressBar(update_interval=10))

In [35]:
trainer.run()

A Jupyter Widget

A Jupyter Widget

In [52]:
exp_sample = d_exp.sample(1000)
obj_sample = d_obj[exp_sample.index]
obj_sample

24276    2
18266    2
51926    0
55128    1
19280    1
59939    2
63158    0
15445    1
3216     1
7579     0
29855    2
12267    1
54043    2
16965    0
63171    1
64106    0
20286    1
40978    2
59332    2
33613    1
16245    1
27919    1
45803    1
56852    0
16051    0
13891    1
62792    2
4232     0
40732    0
56855    2
        ..
4831     2
54925    1
25983    2
20893    1
46186    2
44568    2
22360    2
44599    0
1594     1
58776    2
18323    1
38700    1
43395    0
47789    0
35704    1
57419    0
61380    0
25427    2
8738     1
13996    1
56728    2
14748    2
42764    2
43417    2
12190    0
53408    0
23613    2
59344    0
47426    1
58217    2
Name: price, Length: 1000, dtype: int32

In [53]:
exp_sample_data = xp.array(exp_sample, dtype=xp.float32)
res = model.predictor(exp_sample_data)

In [54]:
result = pd.DataFrame([obj_sample.reset_index().price, chainer.cuda.to_cpu(res.data.argmax(axis=1))]).T
result

,price,Unnamed 0
0,2,2
1,2,1
2,0,1
3,1,0
4,1,2
5,2,2
6,0,1
7,1,1
8,1,2
9,0,0


In [55]:
result[result.price == result['Unnamed 0']].shape

(540, 2)

In [56]:
result[(result.price - result['Unnamed 0']).abs() == 2].shape

(74, 2)

In [57]:
model.predictor

In [58]:
serializers.save_npz('good_predict_500_1000_200_3.npz', model.predictor)